In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import json
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
from pyquery import PyQuery as pq
import requests
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
eq_df = pd.read_csv('tempdata/earthquakes_catalog.csv', sep='|')

In [45]:
"""
#Add column for year
years = []
for time in eq_df['origintime']:
    year = int(str.split(time, '-')[0])
    years.append(year)
    #index = time.key
year_column = pd.Series(years, index=eq_df.index)
eq_df['year'] = year_column
"""

"\n#Add column for year\nyears = []\nfor time in eq_df['origintime']:\n    year = int(str.split(time, '-')[0])\n    years.append(year)\n    #index = time.key\nyear_column = pd.Series(years, index=eq_df.index)\neq_df['year'] = year_column\n"

In [3]:
eq_df.head()

,Unnamed: 0,id,origintime,latitude,longitude,depth,county,origin_src,prefmag,pmag_type,pmag_src,m3hz_ogs,md_ogs,geom,year_float
0,0,255,1980-01-05 07:11:31.21,35.586,-97.894,5,CANADIAN,OGS,1.9,M3Hz,OGS,1.9,1.7,0101000020E6100000F0A7C64B377958C05EBA490C02CB...,1980.130
1,1,256,1980-01-12 07:12:56.45,36.453,-97.642,5,GARFIELD,OGS,1.7,M3Hz,OGS,1.7,1.4,0101000020E61000000C022B87166958C0448B6CE7FB39...,1980.320
2,2,257,1980-02-03 00:46:30.05,33.994,-97.463,5,LOVE,OGS,2.2,M3Hz,OGS,2.2,2.0,0101000020E61000001283C0CAA15D58C0AC1C5A643BFF...,1980.920
3,3,258,1980-02-05 04:32:35.45,34.046,-97.451,5,LOVE,OGS,2.1,M3Hz,OGS,2.1,1.9,0101000020E6100000BE9F1A2FDD5C58C0D9CEF753E305...,1980.980
4,4,259,1980-03-09 03:57:10.56,35.100,-95.100,5,HASKELL,OGS,1.2,M3Hz,OGS,1.2,1.4,0101000020E61000006666666666C657C0CDCCCCCCCC8C...,1980.188


# Miscellaneous things to add to final notebook

#0.1 Additional Imports

In [4]:
import datetime
#import statsmodels.formula.api as sm
import statsmodels.api as sm

#0.2 Poisson Process Code

#0.2.1 No Recent Changes

In [5]:
#function to create list of interarrival times in hours from df 
#1
def get_hours_between(df):
    dates=[]
    origintimes = df.origintime.values
    for date in origintimes:
        year, month, day = date.split('-')
        day, hour = day.split(' ')
        hour, minute, second = hour.split(':')
        if len(second.split('.'))==2:
            second, microsecond = second.split('.')
        elif len(second.split('.'))==1:
            microsecond=0
        dates.append(datetime.datetime(int(year), int(month), int(day), int(hour), int(minute), 
                                       int(second), int(microsecond)))
    dates=sorted(dates)
    deltas=[]
    for i in range(1,len(dates)):
        delta = dates[i] - dates[i-1]
        delta = delta.total_seconds()/3600
        deltas.append(delta)
    deltas = np.array(deltas)
    return deltas

#0.2.2 Recent Changes, Need to add to Final

In [39]:
#2
#fit and plot exponential to data
#k is manual assessment of number of parameters
def fit_expo(deltas, bins=100, xmax=100, ymax=0.02, k=1., force_lambd=None, plot=True):
    #Use Maxmimum Likelihood Estimator to get exponential fit
    expo_fit = sp.stats.expon.fit(deltas, floc=0)
    
    #Calculate Summary Statistics
    k=float(k)
    mean = np.mean(deltas) #sample mean
    if pd.isnull(force_lambd):
        lambd = expo_fit[1] #fitted mean
    else:
        lambd=force_lambd
    n = float(len(deltas))
    ks_d, ks_pval=sp.stats.kstest(deltas, 'expon', alternative='two-sided')
    ppstats = sp.stats.probplot(deltas, dist='expon')
    r = ppstats[1][2]
    r2 = r**2
    log_lik = n*np.log(lambd) - lambd*np.sum(deltas)
    aic = 2*k - 2*log_lik
    ssr = np.sum((deltas-lambd)**2)
    aic2 = n*np.log(ssr/n) + 2*k
    aic_c = aic + float(((2*k)*(k+1))/(n-k-1))
    mse = np.mean((deltas-lambd)**2)
    ad = sp.stats.anderson(deltas, dist='expon')
    ad_stat = ad[0]
    ad_critical_value = ad[1][2]

    
    if plot==True:
        #Overlay fitted exponential over histogram
        plt.hist(deltas, bins=bins, normed=True)
        x = np.arange(0,xmax,0.01)
        y = sp.stats.expon.pdf(x, scale=lambd)
        plt.plot(x,y)
        plt.xlim([0,xmax])
        plt.ylim([0,ymax])

        #QQ Plot for fit to exponential
        sm.qqplot(deltas, dist='expon', fit=True, line='45')
        plt.show()
    
    #Print Summary Statistics
    print "Summary Statistics for Distributional Fit"
    print "lambda = " + str(lambd)
    print "mean(data) = " + str(mean)
    print "sample size = " + str(n)
    print "AIC = " + str(aic) + "??" 
    print "AICc = " + str(aic_c) + "??"
    print "ssr= " + str(ssr) 
    print "AIC2 =" + str(aic2) + "??"
    print "Summary Statistics for Predicton"
    print "MSE = " + str(mse)
    print "####Not Updated for Comparing Models#####"
    print "Kolmogorov-Smirnov Test Statistic D=" + str(ks_d)
    print "Kolmogorov-Smirnov Test p-value=" + str(ks_pval)
    print "QQ R^2 = " + str(r2)
    print "AD Stat = " + str(ad_stat)
    print "AD Critical Value = " + str(ad_critical_value)
    return(lambd, mean, n, ks_d, ks_pval, r2, aic)

In [46]:
#3
#get interarrival times for whole df
def show_expo_fit(df, bins=40, xmax=10000, ymax=0.001, k=1., force_lambd=None, plot=True):
    deltas = get_hours_between(df)
    fit = fit_expo(deltas, bins=bins, xmax=xmax, ymax=ymax, k=k, force_lambd=force_lambd, plot=plot)
    lambd = fit[0]
    return lambd

Add Chi-Squared Goodness of Fit Test http://www.itl.nist.gov/div898/handbook/eda/section3/eda35f.htm

Note: Could also fit to Poisson

Note: Look into "fit" algorithm in scipy, statsmodels

Note: Is this the standard plot for showing distributional fit? 

Note: Need to look into understanding QQ Plots and Probability Plots

Use AIC for comparing distributional fit.

    *Starting Point: Given taht this is the correct distribution, what is the log-likelihood of the data?
    
    *if the number of data points is small, then some correction is often necessary (see AICc, below).
    
    *\mathrm{AIC} = 2k - 2\ln(L)
    *Important websites: http://www.easydatascience.com
    *http://stats.stackexchange.com/questions/31768/is-it-okay-to-compare-fitted-distributions-with-the-aic

Look into nonparametric methods for simulating the distribution here.

Look into this further: http://www.geos.ed.ac.uk/homes/abell5/Modelcode.html

Power Law as a Model for Earthquakes

#1 Baseline fit is not good

In [40]:
lambd1 = show_expo_fit(eq_df, plot=False)

Summary Statistics for Distributional Fit
lambda = 16.6940496181
mean(data) = 16.5797702164
sample size = 18990.0
AIC = 10405323.8944??
AICc = 10405323.8947??
ssr= 139442083.663
AIC2 =169041.334947??
Summary Statistics for Predicton
MSE = 7342.92173052
####Not Updated for Comparing Models#####
Kolmogorov-Smirnov Test Statistic D=0.243090380217
Kolmogorov-Smirnov Test p-value=0.0
QQ R^2 = 0.448214169706
AD Stat = inf
AD Critical Value = 1.341


#2 Splitting at 2010 is better: pre-2010 looks like Poisson Process

This is how well the previous model predicted the data before 2010.

In [47]:
lambd2 = show_expo_fit(eq_df[eq_df.year_float<2010], force_lambd=lambd1, plot=False, k=1.)

Summary Statistics for Distributional Fit
lambda = 16.6940496181
mean(data) = 156.831564075
sample size = 1676.0
AIC = 8766617.85873??
AICc = 8766617.86112??
ssr= 135454137.015
AIC2 =18940.7637778??
Summary Statistics for Predicton
MSE = 80819.8908201
####Not Updated for Comparing Models#####
Kolmogorov-Smirnov Test Statistic D=0.702367579658
Kolmogorov-Smirnov Test p-value=0.0
QQ R^2 = 0.959413948951
AD Stat = 556.03939785
AD Critical Value = 1.341


This is the fit when we do split the data

In [51]:
lambd3 = show_expo_fit(eq_df[eq_df.year_float<2010], plot=False, k=2.)

Summary Statistics for Distributional Fit
lambda = 156.831519794
mean(data) = 156.831564075
sample size = 1676.0
AIC = 82429295.3557??
AICc = 82429295.3628??
ssr= 102539972.537
AIC2 =18476.1982069??
Summary Statistics for Predicton
MSE = 61181.3678621
####Not Updated for Comparing Models#####
Kolmogorov-Smirnov Test Statistic D=0.702367579658
Kolmogorov-Smirnov Test p-value=0.0
QQ R^2 = 0.959413948951
AD Stat = 556.03939785
AD Critical Value = 1.341


#Exponential Fit

In [41]:
#Exponential Regression
x = np.arange(1,100,1)
y = np.random.poisson(x)
df = pd.DataFrame({'x': x, 'y': y}, index=range(1,100))

exponential_model = sm2.GLM(y,x, sm.families.family.Poisson(sm.genmod.families.links.identity))
exp_model = exponential_model.fit()
print exp_model.summary()
plt.plot(x,y)

#ols_model = sm.ols(formula = 'y~x', data = df).fit()
#print ols_model.summary()
#df.head()

AttributeError: 'module' object has no attribute 'families'

In [184]:
sm.families.family.Poisson.links

[statsmodels.genmod.families.links.log,
 statsmodels.genmod.families.links.identity,
 statsmodels.genmod.families.links.sqrt]